# GTEx model with different priors

## Load libraries

In [1]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

# 3. Install PLIER2 (mchikina/PLIER2) if not already installed
if (!requireNamespace("PLIER2", quietly = TRUE)) {
    REPO_PATH <- "/home/msubirana/Documents/pivlab/PLIER2"  # adjust
    remotes::install_local(REPO_PATH, force = TRUE, dependencies = FALSE)
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER2)
library(clusterProfiler)
library(org.Hs.eg.db)
library(dplyr)


source(here("config.R"))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses



clusterProfiler v4.10.0  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:stats’:

    filter


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGener

## Output directory

In [2]:
output_data_dir <- config$GTEx$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

In [3]:
options(repr.matrix.max.rows = Inf)

gtex_plier2_kegg <- readRDS(file.path(output_data_dir, "gtex_KEGG_2021_Human_PLIER2.rds"))

In [4]:
kegg_plier2_summary <- gtex_plier2_kegg$summary %>%
    dplyr::filter(FDR < 0.05 & AUC > 0.7) %>%
    dplyr::group_by(LV) %>%
    dplyr::summarise(pathway = paste(pathway, collapse = ', '), .groups = "drop") %>%
    dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

In [5]:
gtex_baseRes <- readRDS(file.path(output_data_dir, "gtex_PLIER2_baseRes.rds"))

In [6]:
gtex_genes <- readRDS(file.path(output_data_dir, "gtex_genes.rds"))

In [7]:
colnames(gtex_baseRes$Z) <- paste0('LV', seq_len(ncol(gtex_baseRes$Z)))
rownames(gtex_baseRes$Z) <- gtex_genes

In [8]:
head(gtex_baseRes$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV197,LV198,LV199,LV200,LV201,LV202,LV203,LV204,LV205,LV206
WASH7P,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.0000000,0,0,0.1239626,0.0000000,0.0000000
RP11-34P13.15,0,0,0,0,0,0.3678263,0,0,0,0,⋯,0,0,0,0,0.3107976,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.16,0,0,0,0,0,0.3559246,0,0,0,0,⋯,0,0,0,0,0.3337613,0,0,0.0000000,0.0000000,0.0000000
RP11-34P13.18,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.2021179,0,0,0.2418167,0.0000000,0.0000000
AP006222.2,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.3997058,0,0,0.0000000,0.1570023,0.0000000
MTND1P23,0,0,0,0,0,0.0000000,0,0,0,0,⋯,0,0,0,0,0.4758745,0,0,0.0000000,0.0000000,0.4125261


In [9]:
kegg_plier2_Z <- data.frame(gtex_baseRes$Z)

In [10]:
Z <- gtex_baseRes$Z

map <- bitr(rownames(Z), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db)
map <- map[!duplicated(map$SYMBOL), ]

lv_names <- colnames(Z)

top_syms <- setNames(vector("list", length(lv_names)), lv_names)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(rownames(Z), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db):
“20.4% of input gene IDs are fail to map...”


In [11]:
head(top_syms$LV1)

NULL

In [12]:
kegg_plier2_Z %>% 
dplyr::select(LV1)  %>% 
arrange(desc(LV1)) %>%
tail(10)

,LV1
,<dbl>
MT-ND4,0
MT-TH,0
MT-TS2,0
MT-TL2,0
MT-ND5,0
MT-ND6,0
MT-TE,0
MT-CYB,0
MT-TT,0


In [13]:
# for (lv in lv_names) {
#   v <- Z[, lv]
#   idx <- which(!is.na(v) & v > 0)
#   if (length(idx) == 0L) { top_syms[[lv]] <- character(0); next }
#   ord <- idx[order(v[idx], decreasing = TRUE)]
#   top_syms[[lv]] <- rownames(Z)[ord]
# }

# top_entrez <- lapply(top_syms, function(syms) {
#   ids <- map$ENTREZID[match(syms, map$SYMBOL)]
#   unique(as.character(ids[!is.na(ids)]))
# })

# kegg_list <- lapply(top_entrez, function(ids) {
#   if (!length(ids)) return(NULL)
#   enrichKEGG(gene = ids, organism = "hsa", qvalueCutoff = 0.05)
# })

In [14]:
for (lv in lv_names) {
  v <- Z[, lv]
  ord <- order(v, decreasing = TRUE, na.last = NA)
  n_top <- max(1L, ceiling(length(ord) * 0.05))
  top_syms[[lv]] <- rownames(Z)[ord][seq_len(n_top)]
}

top_entrez <- lapply(top_syms, function(syms) {
  ids <- map$ENTREZID[match(syms, map$SYMBOL)]
  unique(as.character(ids[!is.na(ids)]))
})

kegg_list <- lapply(top_entrez, function(ids) {
  if (!length(ids)) return(NULL)
  enrichKEGG(gene = ids, organism = "hsa", qvalueCutoff = 0.05)
})

Reading KEGG annotation online: "https://rest.kegg.jp/link/hsa/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/hsa"...



In [15]:
kegg_list$LV15@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Peroxisome"                                  
 [2] "Fatty acid degradation"                      
 [3] "Complement and coagulation cascades"         
 [4] "Glycine, serine and threonine metabolism"    
 [5] "Valine, leucine and isoleucine degradation"  
 [6] "Tryptophan metabolism"                       
 [7] "One carbon pool by folate"                   
 [8] "Carbon metabolism"                           
 [9] "Metabolism of xenobiotics by cytochrome P450"
[10] "Drug metabolism - cytochrome P450"           
[11] "PPAR signaling pathway"                      
[12] "Biosynthesis of cofactors"                   
[13] "Tyrosine metabolism"                         
[14] "Fatty acid metabolism"                       
[15] "Chemical carcinogenesis - DNA adducts"       
[16] "Pyruvate metabolism"                         
[17] "beta-Alanine metabolism"                     
[18] "Retinol metabolism"                          
[19] "Cholesterol metabolism"                      
[20] "Cysteine and methionine metabolism"          
[21] "Folate transport and metabolism"             
[22] "Vitamin digestion and absorption"            
[23] "Glyoxylate and dicarboxylate metabolism"     
[24] "Butanoate metabolism"                        
[25] "Histidine metabolism"                        
[26] "Arginine biosynthesis"                       
[27] "Arginine and proline metabolism"             
[28] "Alcoholic liver disease"                     
[29] "Alanine, aspartate and glutamate metabolism" 
[30] "Biosynthesis of amino acids"                 
[31] "Glycolysis / Gluconeogenesis"                
[32] "Propanoate metabolism"                       
[33] "Primary bile acid biosynthesis"              
[34] "Drug metabolism - other enzymes"             
[35] "Pantothenate and CoA biosynthesis"           
[36] "Steroid hormone biosynthesis"                
[37] "Lysine degradation"                          
[38] "Glycerolipid metabolism"                     
[39] "2-Oxocarboxylic acid metabolism"             
[40] "Ferroptosis"                                 
[41] "Biosynthesis of unsaturated fatty acids"     
[42] "Lipoic acid metabolism"                      
[43] "Pentose and glucuronate interconversions"    
[44] "Biosynthesis of nucleotide sugars"           
[45] "Amino sugar and nucleotide sugar metabolism" 
[46] "Nicotinate and nicotinamide metabolism"      
[47] "Citrate cycle (TCA cycle)"                   
[48] "Ascorbate and aldarate metabolism"           
[49] "Antifolate resistance"                       
[50] "Proximal tubule bicarbonate reclamation"     
[51] "Bile secretion"                              
[52] "Glutathione metabolism"                      
[53] "Sulfur metabolism"                           
[54] "Taurine and hypotaurine metabolism"          
[55] "Selenocompound metabolism"                   
[56] "Mineral absorption"                          
[57] "Fat digestion and absorption"                
[58] "Arachidonic acid metabolism"                 
[59] "Insulin resistance"                          
[60] "Fatty acid elongation"                       
[61] "Pertussis"                                   
[62] "Steroid biosynthesis"                        
[63] "Staphylococcus aureus infection"             
[64] "Pentose phosphate pathway"                   
[65] "Glucagon signaling pathway"                  
[66] "Galactose metabolism"

In [16]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV15")

LV,pathway
<chr>,<chr>
LV15,"Drug metabolism, Glycine, serine and threonine metabolism, Peroxisome, Retinol metabolism, Tyrosine metabolism"


In [17]:
kegg_list$LV102@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "TNF signaling pathway"                                        
 [2] "Osteoclast differentiation"                                   
 [3] "Pertussis"                                                    
 [4] "Cytokine-cytokine receptor interaction"                       
 [5] "Complement and coagulation cascades"                          
 [6] "Legionellosis"                                                
 [7] "Leishmaniasis"                                                
 [8] "Herpes simplex virus 1 infection"                             
 [9] "Malaria"                                                      
[10] "Epstein-Barr virus infection"                                 
[11] "Lipid and atherosclerosis"                                    
[12] "MAPK signaling pathway"                                       
[13] "Alcoholic liver disease"                                      
[14] "HIF-1 signaling pathway"                                      
[15] "Human T-cell leukemia virus 1 infection"                      
[16] "Staphylococcus aureus infection"                              
[17] "Chagas disease"                                               
[18] "Kaposi sarcoma-associated herpesvirus infection"              
[19] "Toxoplasmosis"                                                
[20] "NF-kappa B signaling pathway"                                 
[21] "AGE-RAGE signaling pathway in diabetic complications"         
[22] "Tuberculosis"                                                 
[23] "Amoebiasis"                                                   
[24] "IL-17 signaling pathway"                                      
[25] "Influenza A"                                                  
[26] "Coronavirus disease - COVID-19"                               
[27] "Inflammatory bowel disease"                                   
[28] "Hematopoietic cell lineage"                                   
[29] "Pathogenic Escherichia coli infection"                        
[30] "Human cytomegalovirus infection"                              
[31] "Rheumatoid arthritis"                                         
[32] "Cellular senescence"                                          
[33] "Apoptosis"                                                    
[34] "JAK-STAT signaling pathway"                                   
[35] "Toll-like receptor signaling pathway"                         
[36] "Phagosome"                                                    
[37] "Salmonella infection"                                         
[38] "PI3K-Akt signaling pathway"                                   
[39] "Transcriptional misregulation in cancer"                      
[40] "Th17 cell differentiation"                                    
[41] "Viral protein interaction with cytokine and cytokine receptor"
[42] "Fluid shear stress and atherosclerosis"                       
[43] "Natural killer cell mediated cytotoxicity"                    
[44] "NOD-like receptor signaling pathway"                          
[45] "Efferocytosis"                                                
[46] "Pancreatic cancer"                                            
[47] "p53 signaling pathway"                                        
[48] "Chronic myeloid leukemia"                                     
[49] "Measles"                                                      
[50] "Ferroptosis"                                                  
[51] "Hepatitis B"                                                  
[52] "TGF-beta signaling pathway"                                   
[53] "African trypanosomiasis"                                      
[54] "Cell adhesion molecules"                                      
[55] "C-type lectin receptor signaling pathway"                     
[56] "Shigellosis"                                                  
[57] "Acute myeloid leukemia"                                       
[58] "Fc gamma R-mediated phagocytosis"                             

In [18]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV102")

LV,pathway
<chr>,<chr>
LV102,"Complement and coagulation cascades, Systemic lupus erythematosus"


In [19]:
kegg_list$LV12@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Shigellosis"                                               
 [2] "Salmonella infection"                                      
 [3] "Endocytosis"                                               
 [4] "Yersinia infection"                                        
 [5] "Pathogenic Escherichia coli infection"                     
 [6] "Osteoclast differentiation"                                
 [7] "Fc gamma R-mediated phagocytosis"                          
 [8] "NOD-like receptor signaling pathway"                       
 [9] "Tuberculosis"                                              
[10] "Phagosome"                                                 
[11] "Chemokine signaling pathway"                               
[12] "Regulation of actin cytoskeleton"                          
[13] "Human immunodeficiency virus 1 infection"                  
[14] "Epstein-Barr virus infection"                              
[15] "Natural killer cell mediated cytotoxicity"                 
[16] "Neutrophil extracellular trap formation"                   
[17] "Lipid and atherosclerosis"                                 
[18] "Human cytomegalovirus infection"                           
[19] "Leukocyte transendothelial migration"                      
[20] "Platelet activation"                                       
[21] "Herpes simplex virus 1 infection"                          
[22] "Bacterial invasion of epithelial cells"                    
[23] "Leishmaniasis"                                             
[24] "Kaposi sarcoma-associated herpesvirus infection"           
[25] "Measles"                                                   
[26] "Legionellosis"                                             
[27] "Toll-like receptor signaling pathway"                      
[28] "Necroptosis"                                               
[29] "Epithelial cell signaling in Helicobacter pylori infection"
[30] "Proteoglycans in cancer"                                   
[31] "C-type lectin receptor signaling pathway"                  
[32] "Autophagy - animal"                                        
[33] "Rap1 signaling pathway"                                    
[34] "B cell receptor signaling pathway"                         
[35] "Human T-cell leukemia virus 1 infection"                   
[36] "Influenza A"                                               
[37] "Fc epsilon RI signaling pathway"                           
[38] "Viral carcinogenesis"                                      
[39] "Hepatitis B"                                               
[40] "Malaria"                                                   
[41] "Human papillomavirus infection"                            
[42] "Hepatitis C"                                               
[43] "Tight junction"                                            
[44] "Pertussis"                                                 
[45] "PD-L1 expression and PD-1 checkpoint pathway in cancer"    
[46] "Sphingolipid signaling pathway"                            
[47] "Lysosome"                                                  
[48] "Adherens junction"                                         
[49] "Rheumatoid arthritis"                                      
[50] "Acute myeloid leukemia"                                    
[51] "Ferroptosis"                                               
[52] "Neurotrophin signaling pathway"                            
[53] "Renal cell carcinoma"                                      
[54] "Toxoplasmosis"                                             
[55] "Chagas disease"                                            
[56] "Proteasome"                                                
[57] "MAPK signaling pathway"                                    
[58] "Glycolysis / Gluconeogenesis"                              
[59] "Pancreatic cancer"                                         
[60] "Chronic myeloid leukemia"                                  
[61] "Fluid shear stress and atherosclero

In [20]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV12")

LV,pathway
<chr>,<chr>
LV12,"Natural killer cell mediated cytotoxicity, Neutrophil extracellular trap formation, Osteoclast differentiation"


In [21]:
kegg_list$LV117@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "MAPK signaling pathway"     "Estrogen signaling pathway"

In [22]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV117")

LV,pathway
<chr>,<chr>


In [23]:
kegg_list$LV23@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

kegg_plier2_summary  %>%  dplyr::filter(LV == "LV23")

[1] "Coronavirus disease - COVID-19"             
[2] "Pancreatic secretion"                       
[3] "Protein processing in endoplasmic reticulum"
[4] "Non-alcoholic fatty liver disease"          
[5] "Protein digestion and absorption"

LV,pathway
<chr>,<chr>
LV23,"Legionellosis, Pancreatic secretion, Ribosome"


In [24]:
kegg_list$LV44@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

kegg_plier2_summary  %>%  dplyr::filter(LV == "LV44")

[1] "Fatty acid degradation"                                  
 [2] "Fatty acid metabolism"                                   
 [3] "PPAR signaling pathway"                                  
 [4] "Glycine, serine and threonine metabolism"                
 [5] "Calcium signaling pathway"                               
 [6] "Carbon metabolism"                                       
 [7] "beta-Alanine metabolism"                                 
 [8] "Propanoate metabolism"                                   
 [9] "Peroxisome"                                              
[10] "Alanine, aspartate and glutamate metabolism"             
[11] "Hippo signaling pathway"                                 
[12] "GABAergic synapse"                                       
[13] "Phospholipase D signaling pathway"                       
[14] "cAMP signaling pathway"                                  
[15] "Lysine degradation"                                      
[16] "Basal cell carcinoma"                                    
[17] "Biosynthesis of unsaturated fatty acids"                 
[18] "Proteoglycans in cancer"                                 
[19] "Valine, leucine and isoleucine degradation"              
[20] "Axon guidance"                                           
[21] "Dilated cardiomyopathy"                                  
[22] "One carbon pool by folate"                               
[23] "Thyroid hormone signaling pathway"                       
[24] "Arrhythmogenic right ventricular cardiomyopathy"         
[25] "Nitrogen metabolism"                                     
[26] "Morphine addiction"                                      
[27] "Signaling pathways regulating pluripotency of stem cells"
[28] "Fatty acid elongation"                                   
[29] "HIF-1 signaling pathway"                                 
[30] "Mineral absorption"                                      
[31] "Notch signaling pathway"                                 
[32] "Neuroactive ligand signaling"                            
[33] "Parathyroid hormone synthesis, secretion and action"     
[34] "Cysteine and methionine metabolism"                      
[35] "Gastric acid secretion"                                  
[36] "Cytoskeleton in muscle cells"                            
[37] "Wnt signaling pathway"                                   
[38] "Tryptophan metabolism"                                   
[39] "Proximal tubule bicarbonate reclamation"                 
[40] "Glycolysis / Gluconeogenesis"                            
[41] "Salivary secretion"                                      
[42] "Adrenergic signaling in cardiomyocytes"                  
[43] "Pyruvate metabolism"                                     
[44] "Hypertrophic cardiomyopathy"                             
[45] "Insulin secretion"                                       
[46] "Inositol phosphate metabolism"                           
[47] "Melanogenesis"                                           
[48] "Butanoate metabolism"                                    
[49] "Glutamatergic synapse"                                   
[50] "Biosynthesis of amino acids"                             
[51] "Arginine and proline metabolism"

LV,pathway
<chr>,<chr>
LV44,"Alanine, aspartate and glutamate metabolism, Alcoholism, GABAergic synapse"
